<a href="https://colab.research.google.com/github/VirtualGoat/Enhanced-CCTV-security-using-DL/blob/master/videoajay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive/')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
model_path = "/content/drive/My Drive/Colab Notebooks/ImageAI/resnet50_coco_best_v2.0.1.h5"
input_path = "/content/drive/My Drive/Colab Notebooks/ImageAI/a.jpg"
output_path = "/content/drive/My Drive/Colab Notebooks/ImageAI/a2.jpg"


In [10]:
# USAGE
# python yolo.py --image images/baggage_claim.jpg --yolo yolo-coco

# import the necessary packages
import numpy as np
#import argparse
import time
import cv2
import os

# construct the argument parse and parse the arguments
"""
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
ap.add_argument("-y", "--yolo", required=True,
	help="base path to YOLO directory")
ap.add_argument("-c", "--confidence", type=float, default=0.5,
	help="minimum probability to filter weak detections")
ap.add_argument("-t", "--threshold", type=float, default=0.3,
	help="threshold when applyong non-maxima suppression")
args = vars(ap.parse_args())
"""
args = {}

args['image'] = '/content/drive/My Drive/Colab Notebooks/ImageAI/airport.mp4'
args['yolo'] = '/content/drive/My Drive/Colab Notebooks/ImageAI'
args['confidence'] = 0.2
args['threshold'] = 0.5
# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([args["yolo"], "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([args["yolo"], "yolov3.weights"])
configPath = os.path.sep.join([args["yolo"], "yolov3.cfg"])

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
out = cv2.VideoWriter('output.avi', -1, 20.0, (640,480))
# load our input image and grab its spatial dimensions
cap = cv2.VideoCapture(args["image"])
while True:
    ret, image = cap.read()
    (H, W) = image.shape[:2]
    
    # determine only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    # construct a blob from the input image and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes and
    # associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
    	swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    
    # show timing information on YOLO
    print("[INFO] YOLO took {:.6f} seconds".format(end - start))
    
    # initialize our lists of detected bounding boxes, confidences, and
    # class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []
    
    # loop over each of the layer outputs
    for output in layerOutputs:
    	# loop over each of the detections
    	for detection in output:
    		# extract the class ID and confidence (i.e., probability) of
    		# the current object detection
    		scores = detection[5:]
    		classID = np.argmax(scores)
    		confidence = scores[classID]
    
    		# filter out weak predictions by ensuring the detected
    		# probability is greater than the minimum probability
    		if confidence > args["confidence"]:
    			# scale the bounding box coordinates back relative to the
    			# size of the image, keeping in mind that YOLO actually
    			# returns the center (x, y)-coordinates of the bounding
    			# box followed by the boxes' width and height
    			box = detection[0:4] * np.array([W, H, W, H])
    			(centerX, centerY, width, height) = box.astype("int")
    
    			# use the center (x, y)-coordinates to derive the top and
    			# and left corner of the bounding box
    			x = int(centerX - (width / 2))
    			y = int(centerY - (height / 2))
    
    			# update our list of bounding box coordinates, confidences,
    			# and class IDs
    			boxes.append([x, y, int(width), int(height)])
    			confidences.append(float(confidence))
    			classIDs.append(classID)
    
    # apply non-maxima suppression to suppress weak, overlapping bounding
    # boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, args["confidence"],
    	args["threshold"])
    
    # ensure at least one detection exists
    count = 0
    if len(idxs) > 0:
    	# loop over the indexes we are keeping
    	for i in idxs.flatten():
    		# extract the bounding box coordinates
    		(x, y) = (boxes[i][0], boxes[i][1])
    		(w, h) = (boxes[i][2], boxes[i][3])
            
    		if LABELS[classIDs[i]]=='person':
    			count += 1    
                
    		# draw a bounding box rectangle and label on the image
    		color = [int(c) for c in COLORS[classIDs[i]]]
    		cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
    		cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
    			0.5, color, 2)
    
     #show the output image
    # cv2.putText(image, str(count), (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    # cv2.imshow("Video", image)
    frame = cv2.flip(image,0)

        # write the flipped frame
    out.write(frame)
    if(cv2.waitKey(1) & 0xFF==ord('q')):
        break
    
cap.release()
cv2.destroyAllWindows()


[INFO] loading YOLO from disk...
[INFO] YOLO took 1.758638 seconds
[INFO] YOLO took 1.777475 seconds
[INFO] YOLO took 1.612000 seconds
[INFO] YOLO took 1.641638 seconds
[INFO] YOLO took 1.587275 seconds
[INFO] YOLO took 1.595597 seconds
[INFO] YOLO took 1.612997 seconds
[INFO] YOLO took 1.590325 seconds
[INFO] YOLO took 1.593331 seconds
[INFO] YOLO took 1.632062 seconds
[INFO] YOLO took 1.649592 seconds
[INFO] YOLO took 1.580294 seconds
[INFO] YOLO took 1.587043 seconds
[INFO] YOLO took 1.588024 seconds
[INFO] YOLO took 1.611794 seconds
[INFO] YOLO took 1.583399 seconds
[INFO] YOLO took 1.600291 seconds
[INFO] YOLO took 1.588390 seconds
[INFO] YOLO took 1.613704 seconds
[INFO] YOLO took 1.587095 seconds
[INFO] YOLO took 1.596680 seconds
[INFO] YOLO took 1.609923 seconds
[INFO] YOLO took 1.783138 seconds
[INFO] YOLO took 1.585858 seconds
[INFO] YOLO took 1.592028 seconds
[INFO] YOLO took 1.582035 seconds
[INFO] YOLO took 1.607314 seconds
[INFO] YOLO took 1.619901 seconds
[INFO] YOLO too

AttributeError: ignored